# Compbio analysis

In [ ]:
import lamindb as ln
import scanpy as sc

ln.nb.header()

In [ ]:
!lndb login test-user2

## Link experimental metadata to the datasets

Query for datasets ingested from a bioinformatics run:

In [ ]:
bfx_runs = ln.db.query.pipeline_run()
bfx_runs

In [ ]:
dtransform_id = ln.db.query.dtransform(pipeline_run_id=bfx_runs[0].id)[0].id
dobjects = ln.db.query.dobject(dtransform_id=dtransform_id)
len(dobjects)

Now let's link biometa to the bfx run output datasets:

In [ ]:
for dobject in dobjects:
    ln.db.link.biometa(dobject_id=dobject.id, biometa_id=2)

Let's also link the screen results to its biometa.

In [ ]:
dobjects = ln.db.query.dobject(name="schmidt22-crispra-gws-IFNG")
dobjects

In [ ]:
ln.db.link.biometa(dobject_id=dobjects[0].id, biometa_id=1)

Fill out additional biometa fields for dobjects ingested with feature models.

In [ ]:
ln.db.query.jupynb(name="Track and analyze experimental data")

In [ ]:
jupynb_id = ln.db.query.jupynb(name="Track and analyze experimental data")[-1].id
dtransform_id = ln.db.query.dtransform(jupynb_id=jupynb_id)[-1].id

In [ ]:
dobjects = ln.db.query.dobject(dtransform_id=dtransform_id)

In [ ]:
dobjects

In [ ]:
biometa1 = ln.db.query.biometa(id=1)[0]

for dobject in dobjects:
    doject_biometa = ln.db.query.dobject_biometa(dobject_id=dobject.id)
    assert len(doject_biometa) == 1
    biometa_id = doject_biometa[0].biometa_id
    biometa = ln.db.query.biometa(id=biometa_id)[0]
    insert_dict = {}
    for key in biometa.dict().keys():
        if key == "id":
            continue
        if biometa.__getattribute__(key) is not None:
            insert_dict[key] = biometa1.__getattribute__(key)

    ln.db.update.biometa(key=biometa.id, **insert_dict)

## Load in the raw data generated from the BFX pipeline

In [ ]:
h5ads = ln.db.query.dobject(file_suffix=".h5ad")
h5ads

In [ ]:
hits = ln.db.query.dobject(name="schmidt22_crispra_gws_IFNG_hits")
hits

In [ ]:
adata = ln.db.load(h5ads[0])

In [ ]:
screen_hits = ln.db.load(hits[0])

## Perform single cell analysis, integrating the CRISPR screen data

In [ ]:
sc.tl.score_genes(
    adata, adata.var_names.intersection(screen_hits["id"].values).tolist()
)

In [ ]:
sc.pl.umap(adata, color="cluster_name")

In [ ]:
sc.pl.umap(adata, color="score")

In [ ]:
sc.pl.matrixplot(adata, groupby="cluster_name", var_names=["score"])

## Ingest the processed dataset into LaminDB

In [ ]:
ln.db.ingest.add(adata, name="schmidt22_perturbseq_analyzed")

In [ ]:
ln.db.ingest.commit()